In [1]:
%matplotlib inline
import os
import glob
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [ ]:
# This data is not currently publicly available

In [2]:
#indir = '../data/annotated_datasets/'
indir = '../../human-eval-analysis/data/nucleus/human_eval_data/'
responses = ["Invalid", "Rare", "Specific", "Average", "Typical", "Very Typical"]
response_dict = dict(zip(responses, range(len(responses))))

In [4]:
models = ['k=40_i=0.1', 'k=40_t=0.7_i=0.1', 'k=640_i=0.1', 'p=0.95_i=0.1', 't=0.9', 't=1']
dfs = []

for m_i, model in enumerate(models):
    infile = indir + 'human_' + model + '.csv'
    df = pd.read_csv(infile, header=0, index_col=None)
    # assign a model ID combining type and setting
    df['model'] = 'human'

    # The human data is just duplicated exactly in each file, so only append it once
    if m_i == 0:
        dfs.append(df)
    
    vals = []
    for j in range(25):
        vals.extend([response_dict[val] for val in df['Answer.' + str(j)].values])
    
    infile = indir + 'machine_' + model + '.csv'
    df = pd.read_csv(infile, header=0, index_col=None)
    df['model'] = 'machine_' + model
    dfs.append(df)
    
    vals = []
    for j in range(25):
        vals.extend([response_dict[val] for val in df['Answer.' + str(j)].values])

    df = pd.concat(dfs, axis=0, ignore_index=True)    
print(df.shape)
df.tail()

(1120, 330)


,Answer.24,Input.CTX24,Input.LOGP24,Input.TYPE24,Input.IDX24,Answer.23,Input.CTX23,Input.LOGP23,Input.TYPE23,Input.IDX23,Answer.22,Input.CTX22,Input.LOGP22,Input.TYPE22,Input.IDX22,Answer.21,Input.CTX21,Input.LOGP21,Input.TYPE21,Input.IDX21,Answer.20,Input.CTX20,Input.LOGP20,Input.TYPE20,Input.IDX20,Answer.19,Input.CTX19,Input.LOGP19,Input.TYPE19,Input.IDX19,Answer.18,Input.CTX18,Input.LOGP18,Input.TYPE18,Input.IDX18,Answer.17,Input.CTX17,Input.LOGP17,Input.TYPE17,Input.IDX17,Answer.16,Input.CTX16,Input.LOGP16,Input.TYPE16,Input.IDX16,Answer.15,Input.CTX15,Input.LOGP15,Input.TYPE15,Input.IDX15,Answer.14,Input.CTX14,Input.LOGP14,Input.TYPE14,Input.IDX14,Answer.13,Input.CTX13,Input.LOGP13,Input.TYPE13,Input.IDX13,Answer.12,Input.CTX12,Input.LOGP12,Input.TYPE12,Input.IDX12,Answer.11,Input.CTX11,Input.LOGP11,Input.TYPE11,Input.IDX11,Answer.10,Input.CTX10,Input.LOGP10,Input.TYPE10,Input.IDX10,Answer.9,Input.CTX9,Input.LOGP9,Input.TYPE9,Input.IDX9,Answer.8,Input.CTX8,Input.LOGP8,Input.TYPE8,Input.IDX8,Answer.7,Input.CTX7,Input.LOGP7,Input.TYPE7,Input.IDX7,Answer.6,Input.CTX6,Input.LOGP6,Input.TYPE6,Input.IDX6,Answer.5,Input.CTX5,Input.LOGP5,Input.TYPE5,Input.IDX5,Answer.4,Input.CTX4,Input.LOGP4,Input.TYPE4,Input.IDX4,Answer.3,Input.CTX3,Input.LOGP3,Input.TYPE3,Input.IDX3,Answer.2,Input.CTX2,Input.LOGP2,Input.TYPE2,Input.IDX2,Answer.1,Input.CTX1,Input.LOGP1,Input.TYPE1,Input.IDX1,Answer.0,Input.CTX0,Input.LOGP0,Input.TYPE0,Input.IDX0,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,AutoApprovalDelayInSeconds,Expiration,NumberOfSimilarHITs,LifetimeInSeconds,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,RejectionTime,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.OUT0,Input.OUT1,Input.OUT2,Input.OUT3,Input.OUT4,Input.OUT5,Input.OUT6,Input.OUT7,Input.OUT8,Input.OUT9,Input.OUT10,Input.OUT11,Input.OUT12,Input.OUT13,Input.OUT14,Input.OUT15,Input.OUT16,Input.OUT17,Input.OUT18,Input.OUT19,Input.OUT20,Input.OUT21,Input.OUT22,Input.OUT23,Input.OUT24,Answer.0.Average,Answer.0.Invalid,Answer.0.Rare,Answer.0.Specific,Answer.0.Typical,Answer.0.Very Typical,Answer.1.Average,Answer.1.Invalid,Answer.1.Rare,Answer.1.Specific,Answer.1.Typical,Answer.1.Very Typical,Answer.10.Average,Answer.10.Invalid,Answer.10.Rare,Answer.10.Specific,Answer.10.Typical,Answer.10.Very Typical,Answer.11.Average,Answer.11.Invalid,Answer.11.Rare,Answer.11.Specific,Answer.11.Typical,Answer.11.Very Typical,Answer.12.Average,Answer.12.Invalid,Answer.12.Rare,Answer.12.Specific,Answer.12.Typical,Answer.12.Very Typical,Answer.13.Average,Answer.13.Invalid,Answer.13.Rare,Answer.13.Specific,Answer.13.Typical,Answer.13.Very Typical,Answer.14.Average,Answer.14.Invalid,Answer.14.Rare,Answer.14.Specific,Answer.14.Typical,Answer.14.Very Typical,Answer.15.Average,Answer.15.Invalid,Answer.15.Rare,Answer.15.Specific,Answer.15.Typical,Answer.15.Very Typical,Answer.16.Average,Answer.16.Invalid,Answer.16.Rare,Answer.16.Specific,Answer.16.Typical,Answer.16.Very Typical,Answer.17.Average,Answer.17.Invalid,Answer.17.Rare,Answer.17.Specific,Answer.17.Typical,Answer.17.Very Typical,Answer.18.Average,Answer.18.Invalid,Answer.18.Rare,Answer.18.Specific,Answer.18.Typical,Answer.18.Very Typical,Answer.19.Average,Answer.19.Invalid,Answer.19.Rare,Answer.19.Specific,Answer.19.Typical,Answer.19.Very Typical,Answer.2.Average,Answer.2.Invalid,Answer.2.Rare,Answer.2.Specific,Answer.2.Typical,Answer.2.Very Typical,Answer.20.Average,Answer.20.Invalid,Answer.20.Rare,Answer.20.Specific,Answer.20.Typical,Answer.20.Very Typical,Answer.21.Average,Answer.21.Invalid,Answer.21.Rare,Answer.21.Specific,Answer.21.Typical,Answer.21.Very Typical,Answer.22.Average,Answer.22.Invalid,Answer.22.Rare,Answer.22.Specific,Answer.22.Typical,Answer.22.Very Typical,Answer.23.Average,Answer.23.Invalid,Answer.23.Rare,Answer.23.Specific,Answer.23.Typical,Answ

In [5]:
worker_counter = Counter(df['WorkerId'].values)
print(len(worker_counter))
worker_dict = dict(zip(sorted(worker_counter), range(len(worker_counter))))

140


In [6]:
text_counter = Counter()
for i in range(25):
    # avoid some nan's in the output
    texts = [text for text in df['Input.OUT' + str(i)].values if type(text) == str]
    text_counter.update(texts)
print(len(text_counter))
text_dict = dict(zip(sorted(text_counter), range(len(text_counter))))

1399


In [7]:
model_counter = Counter()
for i in range(25):
    model_counter.update(df['model'].values)
print(model_counter)
model_dict = dict(zip(sorted(model_counter), range(len(model_counter))))

Counter({'human': 4000, 'machine_k=40_i=0.1': 4000, 'machine_k=40_t=0.7_i=0.1': 4000, 'machine_k=640_i=0.1': 4000, 'machine_p=0.95_i=0.1': 4000, 'machine_t=0.9': 4000, 'machine_t=1': 4000})


In [8]:
rows = []
responses_by_sent_by_model = {model: defaultdict(list) for model in model_dict}

for i in df.index:
    worker = df.loc[i, 'WorkerId']
    worker_id = 'W' + str(worker_dict[worker])
    for j in range(25):
        model = df.loc[i, 'Input.TYPE' + str(j)]
        text = df.loc[i, 'Input.OUT' + str(j)]
        if type(text) == str:
            item_id = 'I' + str(text_dict[text])
            response = df.loc[i, 'Answer.' + str(j)]
            response_val = float(response_dict[response])
            model = df.loc[i, 'model']
            model_id = 'M' + str(model_dict[model])
            rows.append([worker, worker_id, model, model_id, text, item_id, response, response_val])

df_out = pd.DataFrame(rows, columns=['worker', 'worker_id', 'model', 'model_id', 'text', 'text_id', 'response', 'response_val'])
print(df_out.shape)
df_out.head()

(27980, 8)


,worker,worker_id,model,model_id,text,text_id,response,response_val
0,A1F1BIPJR11LSR,W13,human,M0,H.R. 3149 (111th) was a bill in the United Sta...,I442,Average,3.0
1,A1F1BIPJR11LSR,W13,human,M0,"Islamabad, Oct. 9 (ANI): Afghanistan Taliban h...",I596,Typical,4.0
2,A1F1BIPJR11LSR,W13,human,M0,Dr Sarah Wollaston condemns use of police stat...,I359,Very Typical,5.0
3,A1F1BIPJR11LSR,W13,human,M0,"On Friday, the Pentagon reported that 38 soldi...",I756,Typical,4.0
4,A1F1BIPJR11LSR,W13,human,M0,"Psychopaths are everywhere. No really, they're...",I822,Typical,4.0


In [9]:
df_subset = pd.concat([df_out[df_out['model_id'] == 'M0'], df_out[df_out['model_id'] == 'M1']], axis=0, ignore_index=True)
df_subset.shape

(8000, 8)

In [10]:
outdir = os.path.join('..', 'data', 'annotated_datasets')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_out.to_csv(os.path.join(outdir, 'nucleus.csv'))